In [ ]:
import pandas as pd
import numpy as np
import os
import random

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import OrdinalEncoder

In [ ]:
# 파일 경로
file_path = '/content/drive/MyDrive/1데이콘/K리그-서울시립대공개AI경진대회/dataset/'

In [46]:
df = pd.read_csv(file_path + 'process_df.csv')

process_df = df.copy()

# Initial feature selection

In [49]:
def drop_features(df):
  df = df.copy()

  meta_cols = [
      'start_x', 'start_y', 'end_x', 'end_y', 'game_episode','episode_id',
      'game_id','player_id','team_id','period_id','result_name','is_home','is_train','time_seconds'
      ]

  cols = []
  cols.extend(df.columns[df.columns.str.startswith('new_')].tolist())

  used_cols = cols + meta_cols
  used_cols = sorted(list(set(used_cols)))

  df_filtered = df[used_cols].copy()
  print(f"최종 사용 피처 수: {len(used_cols)}")
  print(f"전체 데이터 건수: {len(df_filtered)}")

  return df_filtered

process_df = drop_features(process_df)

최종 사용 피처 수: 64
전체 데이터 건수: 766552


# train test split

In [50]:
def train_test_split(df):
  train = df[df['is_train'] == 1]
  test = df[df['is_train'] == 0]

  return train, test

In [51]:
train, test = train_test_split(process_df)

# normalize

In [52]:
def cat_normalize(train, test, cat_cols):
  for col in cat_cols:
    oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    train_vals = train[col].astype(str).values.reshape(-1, 1)
    test_vals  = test[col].astype(str).values.reshape(-1, 1)

    oe.fit(train_vals)

    train[col+ '_curr'] = oe.transform(train_vals).reshape(-1).astype(int)
    test[col+ '_curr']  = oe.transform(test_vals).reshape(-1).astype(int)

  return train, test

In [53]:
cat_cols = ["result_name", "team_id", "player_id", 'new_opp_team_id']

train, test = cat_normalize(train, test, cat_cols)

In [54]:
train.memory_usage().sum() / 1024 ** 2,'mb'

(np.float64(375.5760040283203), 'mb')

# Save

In [55]:
train.to_parquet(file_path + "train_df.parquet")
test.to_parquet(file_path  + "test_df.parquet")